<a href="https://colab.research.google.com/github/CodeByIman/CodeByIman/blob/main/createcollonnebrand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import re

# 1. Chargement de ton dataset
# Remplace 'ton_fichier.csv' par le vrai nom de ton fichier
try:
    df = pd.read_csv('data_jumia_cosmetic.csv')
    print("✅ Dataset chargé avec succès !")
except FileNotFoundError:
    print("❌ Erreur : Fichier non trouvé. Vérifie le nom.")



✅ Dataset chargé avec succès !


In [9]:


# 1. LISTE DES MARQUES CLASSIQUES (Comme avant)
known_brands = [
    "La Roche Posay", "The Ordinary", "L'Oréal Paris", "L'Oréal", "L'Oreal",
    "CeraVe", "Vichy", "NIVEA", "Garnier", "Bioderma", "Avene", "SVR",
    "Uriage", "Neutrogena", "Cetaphil", "Eucerin", "Isdin", "Mustela",
    "Mixa", "Dove", "Vaseline", "Nuxe", "Filorga", "Clarins",
    "Cosrx", "Beauty of Joseon", "Skin1004", "Some By Mi", "Dr.Althea",
    "Anua", "Laneige", "Disaar", "Farmasi", "Oriflame", "Balea", "Cien",
    "DCP", "ACM", "Bio Balance", "Fruit of the Wokali", "Racine Vita",
    "Nature Soin", "Bio Oil", "Ecladerm", "Kaline", "Revuele", "Revox",
    "Topiderm", "Argan", "Byphasse", "Calliderm", "Cantu", "Olapp",
    "Clear", "Dettol", "Yves Rocher", "Sensilis", "Dermadic", "St Ives",
    "Herbo Ridouane", "Estee Lauder", "Kiehl's"
]

# 2. LA NOUVEAUTÉ : MAP DES GAMMES (Si le titre contient X, alors la marque est Y)
# C'est ici qu'on règle ton problème "Glycolic Bright"
product_lines_map = {
    "Glycolic Bright": "L'Oréal Paris",
    "Revitalift": "L'Oréal Paris",
    "Hyaluron Expert": "L'Oréal Paris",
    "Bright Complete": "Garnier",
    "Fast Bright": "Garnier",
    "Ambre Solaire": "Garnier",
    "Ultra Doux": "Garnier",
    "Effaclar": "La Roche Posay",
    "Anthelios": "La Roche Posay",
    "Cicaplast": "La Roche Posay",
    "Lipikar": "La Roche Posay",
    "Hydro Boost": "Neutrogena",
    "Spot Controlling": "Neutrogena",
    "Sebium": "Bioderma",
    "Créaline": "Bioderma",
    "Cicalfate": "Avene",
    "Cleanance": "Avene",
    "Hyalurogel": "Mixa",
    "Advanced Snail": "Cosrx",
    "Madagascar Centella": "Skin1004",
    "AHA BHA PHA": "Some By Mi",
    "Pigmentbio": "Bioderma",
    "Atoderm": "Bioderma",
    "Q10": "NIVEA", # Attention, Q10 est utilisé par d'autres, mais surtout Nivea ici
    "Sun Zone": "Oriflame",
    "Optimals": "Oriflame",
    "Love Nature": "Oriflame",
    "Milk & Honey": "Oriflame",
    "Depiwhite": "ACM",
    "Sebionex": "ACM"
}

# Mots bruits
noise_words = [
    "Pack", "Lot", "Kit", "Set", "Offre", "Promo", "Spéciale", "Nouveau", "New",
    "Crème", "Creme", "Gel", "Sérum", "Serum", "Huile", "Masque", "Routine", "Savon",
    "Ecran", "Écran", "Lait", "Baume", "Fluide", "Mousse", "Spray", "Lotion",
    "Brosse", "Appareil", "Machine", "Nettoyant", "Gommage", "Ampoule", "Tonique",
    "Soin", "Traitement", "Stick", "Fond", "Bb", "Cc", "Dd", "Eau", "Beurre",
    "Huiles", "Protege", "Protection", "Autre", "Mini", "Duo", "Trio"
]

# Compilation Regex Marques
brands_sorted = sorted(known_brands, key=len, reverse=True)
brand_pattern = re.compile(r"\b(" + "|".join(map(re.escape, brands_sorted)) + r")\b", re.IGNORECASE)

def extract_brand_advanced(title):
    if not isinstance(title, str):
        return "Inconnu"

    # 1. PRIORITÉ : Vérifier si une Gamme connue est dans le titre
    # (C'est ça qui va sauver ton "Glycolic Bright")
    for gamme, brand in product_lines_map.items():
        if gamme.lower() in title.lower():
            return brand

    # 2. Si pas de gamme, on cherche la Marque explicite
    match = brand_pattern.search(title)
    if match:
        return match.group(0).title()

    # 3. Méthode de secours (Premier mot nettoyé)
    clean_title = re.sub(r'[^\w\s]', '', title)
    words = clean_title.split()

    if not words:
        return "Autre"

    first = words[0].title()
    if first not in noise_words and len(first) > 2:
        return first

    return "Autre"

# Application
df['Brand_Clean'] = df['product_name'].apply(extract_brand_advanced)

# Nettoyage final des variantes
df['Brand_Clean'] = df['Brand_Clean'].replace({
    'La Roche Posay1': 'La Roche Posay',
    "L'oreal": "L'Oréal Paris",
    "L'oréal": "L'Oréal Paris",
    "Loreal": "L'Oréal Paris"
})

# Vérifier si "Glycolic" est bien devenu "L'Oréal Paris"
print(df[df['product_name'].str.contains("Glycolic", case=False, na=False)][['product_name', 'Brand_Clean']].head())

                                           product_name    Brand_Clean
179   L'Oréal Paris Pack Glycolic Bright Sérum Yeux ...  L'Oréal Paris
297   L'Oréal Paris Glycolic Bright Sérum Yeux Éclat...  L'Oréal Paris
2658  L'Oreal CRÈME De Nuit Eclaircissante Glycolic ...  L'Oréal Paris
3199  Ducray KERACNYL GLYCOLIC+ Crème désincrustante...         Ducray
3713      Gel Glycolic Bright Nettoyant Moussant 100 Ml  L'Oréal Paris


In [11]:
# 1. Compter les vraies valeurs manquantes (NaN) - devrait être 0 si le script a marché
nan_count = df['Brand_Clean'].isnull().sum()

# 2. Compter les valeurs "Autre" (Marques non détectées par le script)
# Si tu as utilisé "Inconnu" dans le script précédent, remplace 'Autre' par 'Inconnu' ici
other_count = df[df['Brand_Clean'] == 'Autre'].shape[0]

# 3. Calculer le total et le pourcentage
total_rows = len(df)
success_rate = 100 - ((nan_count + other_count) / total_rows * 100)

print(f"--- Résultat de l'extraction ---")
print(f"Total de lignes : {total_rows}")
print(f"Vraies valeurs vides (NaN) : {nan_count}")
print(f"Marques non détectées ('Autre') : {other_count}")
print(f"--------------------------------")
print(f"✅ Taux de succès : {success_rate:.2f}%")

# Bonus : Voir à quoi ressemblent ces produits "Autre" pour améliorer la liste
if other_count > 0:
    print("\n--- Exemple de produits non détectés ---")
    print(df[df['Brand_Clean'] == 'Autre']['product_name'].head(5))

--- Résultat de l'extraction ---
Total de lignes : 4505
Vraies valeurs vides (NaN) : 0
Marques non détectées ('Autre') : 328
--------------------------------
✅ Taux de succès : 92.72%

--- Exemple de produits non détectés ---
63     El Hikma Sérum Vitamine C Pour Visage (Anti-ri...
64          El Hikma Sérum Vitamine C Pour Visage - 30ML
127          BE A MAN Sérum visage nutrition intense OUD
168                              Sérum Anti-Rides Visage
232    K - SECRET SEOUL 1988 Serum : Retinal Liposome...
Name: product_name, dtype: object


In [12]:
df_autre = df[df['Brand_Clean'] == 'Autre']

print(f"Il y a {len(df_autre)} produits classés comme 'Autre'.")
print("--- Liste des titres non identifiés ---")
for titre in df_autre['product_name'].unique():
    print(titre)

print("\n" + "="*50 + "\n")

Il y a 328 produits classés comme 'Autre'.
--- Liste des titres non identifiés ---
El Hikma Sérum Vitamine C Pour Visage (Anti-rides , Anti-oxydant) - 30ML
El Hikma Sérum Vitamine C Pour Visage - 30ML
BE A MAN Sérum visage nutrition intense OUD
Sérum Anti-Rides Visage
K - SECRET SEOUL 1988 Serum : Retinal Liposome 2% + Black Ginseng - 30 ml
K - SECRET K-Secret, Sérum Seoul 1988, Liposome rétinien 2 % + Ginseng noir, 30 ml
Sérum Collagène Escargot Noir COXIR 50 ml
Crème Visage Rétinol – Anti Rides, Anti Taches et boost Collagène – Éclat Jeunesse – Certifié ONSSA
Pack Flavo-c Ultraglican Sérum Antioxydant Jour + Flavo-c Mela-tonin Réparateur Nuit 48h (10+10ampoules)
La Roche Posay1 MELA B3 SÉRUM CONCENTRÉ INTENSIF ANTI-TACHES ANTI-RÉCIDIVE30ML
La Roche Posay1 Retinol B3 Sérum Concentré Anti-Rides Régénérant Resurfaçant – 30ml
Sérum LOUBAN anti-imperfections éclaircissant
Ca SERUM GLYCOLAGEN BEIGE OPALESCENT
Sérum Collagène à l'Escargot Noir COXIR, 50ml
Dr organic Bioactive Skincare Organ

In [13]:
print(df['price_num'].describe())

count    4505.000000
mean      191.171907
std       196.119629
min        18.900000
25%        89.000000
50%       159.000000
75%       246.000000
max      7000.000000
Name: price_num, dtype: float64


In [14]:
zero_price = df[df['price_num'] <= 0]
print(f"\n⚠️ Nombre de produits avec prix <= 0 : {len(zero_price)}")


⚠️ Nombre de produits avec prix <= 0 : 0


In [15]:
expensive_products = df[df['price_num'] > 2000]
print(f"⚠️ Nombre de produits > 2000 MAD : {len(expensive_products)}")
if len(expensive_products) > 0:
    print(expensive_products[['product_name', 'Brand_Clean', 'price_num']])

⚠️ Nombre de produits > 2000 MAD : 7
                                           product_name   Brand_Clean  \
101   Maroc 4 Products Sérum de vitamine E pour le v...         Maroc   
398   Estee Lauder Advanced Night Repair Synchronize...  Estee Lauder   
407   Estee Lauder Duo Advanced Night Repair Synchro...  Estee Lauder   
441   Guerlain (Abeille Royale Double R Renew & Repa...      Guerlain   
2062  Plantil Argania Maroc Crème d'Argan Hydratante...         Argan   
3077             La Prairie Skin caviar crème luxe yeux         Autre   
3194             La Prairie Skin caviar crème luxe yeux         Autre   

      price_num  
101     2034.45  
398     2399.00  
407     3000.00  
441     2499.00  
2062    7000.00  
3077    3199.00  
3194    2850.00  


In [16]:
# Vérifier si des notes dépassent 5
invalid_ratings = df[df['rating_num'] > 5]
print(f"Notes invalides (>5) : {len(invalid_ratings)}")

# Vérifier la distribution des avis (Est-ce qu'on a beaucoup de produits sans avis ?)
no_reviews = df[df['rating_num'] == 0]
print(f"Produits sans aucune note (0) : {len(no_reviews)} ({len(no_reviews)/len(df)*100:.1f}%)")

Notes invalides (>5) : 0
Produits sans aucune note (0) : 3702 (82.2%)
